In [1]:
import cv2
import numpy as np
from skimage.feature import match_template
import matplotlib.pyplot as plt
from copy import deepcopy
from imutils.object_detection import non_max_suppression # pip install imutils
from PIL import Image


In [6]:
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
pytesseract.image_to_string(Image.open('./A067/A067_header.bmp'))

''

In [61]:
def generate_sections(reference_image,multi_up_image, output_img_path,threshold=0.75,saveit=False):

    reference_gray = cv2.cvtColor(reference_image, cv2.COLOR_BGR2RGB)
    multi_up_gray = cv2.cvtColor(multi_up_image, cv2.COLOR_BGR2RGB)

    result = cv2.matchTemplate(multi_up_gray, reference_gray, cv2.TM_CCOEFF_NORMED)
    print(f"{result.shape} total results found...")

    # threshold = 0.75  # Adjust this threshold as needed
    loc = np.where(result >= threshold)

    # Perform non-maximum suppression.
    # template_h, template_w = reference_gray.shape[:2]
    # rects = []
    # for (x, y) in zip(loc[0], loc[1]):
    #     rects.append((x, y, x + template_w, y + template_h))
    # pick = non_max_suppression(np.array(rects))

    # Define a function to remove duplicates within a specified distance (tolerance)
    def remove_duplicates(points, tolerance, horizontal_tolerance):
        unique_points = []
        unique_indices = []
        for i, (x, y) in enumerate(points):
            is_unique = True
            for j, (x_other, y_other) in enumerate(unique_points):
                if abs(x - x_other) < horizontal_tolerance:
                    is_unique = False
                    break
                distance = np.sqrt((x - x_other) ** 2 + (y - y_other) ** 2)
                if distance <= tolerance:
                    is_unique = False
                    break
            if is_unique:
                unique_points.append((x, y))
                unique_indices.append(i)
        
        return np.array(unique_points), np.array(unique_indices)

    x, y = loc
    print(f"{len(x)} total occurrences found...")

    tolerance = 10  # Adjust this value as needed
    horizontal_tolerance = reference_image.shape[0]//2
    unique_loc, unique_indices = remove_duplicates(np.column_stack((x, y)), tolerance, horizontal_tolerance)
    print(f"{len(unique_indices)} unique found...")

    matched_sections = []
    bb = deepcopy(multi_up_image)
    for pt in unique_loc:
        tl=(0,pt[0])
        br=(multi_up_image.shape[1], pt[0] + reference_image.shape[0])
        bb = cv2.rectangle(bb, tl, br, (0, 0, 255), 3)
        matched_sections.append(multi_up_image[tl[1]:br[1],tl[0]:br[0]] )

    # cv2.imshow("BB", cv2.resize(bb, None, fx=0.15, fy=0.15))
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()

    if (saveit):
        for i, matched_img in enumerate(matched_sections):
            # if matched_img.size != 0:
            cv2.imwrite(f"{output_img_path}/{i}.jpg", matched_img)

    return result, bb, matched_sections, loc, unique_loc

In [62]:
ref = '67'
reference_image = cv2.imread(f'./A0{ref}/SA-A0{ref}.jpg', cv2.IMREAD_COLOR)
multi_up_image = cv2.imread("multi_up.bmp", cv2.IMREAD_COLOR) #f'./A0{ref}/A0{ref}_0.bmp'
reference_image = cv2.resize(reference_image, (830, 1073))

r,b,m,l,u = generate_sections(reference_image, multi_up_image, f'./A0{ref}',threshold=0.8,saveit=False)

(5456, 1653) total results found...
245 total occurrences found...
6 unique found...


In [63]:
u

array([[   7,  692],
       [1097,    2],
       [2194,  141],
       [3264,  279],
       [4355,  417],
       [5455,  554]], dtype=int64)